In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-04-25 17:54:09--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  96.0MB/s    in 0.3s    

2025-04-25 17:54:09 (96.0 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code

In [6]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # Filter for books with at least 50 ratings
  df_ratings_count = pd.DataFrame(df_ratings.groupby('isbn')['rating'].count())
  popular_books = list(df_ratings_count[df_ratings_count['rating'] >= 50].index)
  df_books_filtered = df_books[df_books['isbn'].isin(popular_books)]

  # Filter for users with at least 200 ratings
  df_users_count = pd.DataFrame(df_ratings.groupby('user')['rating'].count())
  prolific_users = list(df_users_count[df_users_count['rating'] >= 200].index)
  df_ratings_filtered = df_ratings[df_ratings['user'].isin(prolific_users)]

  # Merge the filtered DataFrames
  df_books_ratings = pd.merge(df_books_filtered, df_ratings_filtered, on='isbn')

  # Create a pivot table
  book_matrix = df_books_ratings.pivot_table(index='title', columns='user', values='rating').fillna(0)

  # Convert the pivot table to a sparse matrix
  book_sparse = csr_matrix(book_matrix)

  # Create and fit the NearestNeighbors model
  model = NearestNeighbors(metric='cosine', algorithm='brute')
  model.fit(book_sparse)

  # Get the index of the input book
  book_index = book_matrix.index.get_loc(book)

  # Find the nearest neighbors
  distances, indices = model.kneighbors(book_matrix.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

  # Prepare the recommended books list
  recommended_books = [book]
  recommended_books.append([
      (book_matrix.index[indices.flatten()[i]], distances.flatten()[i])
      for i in range(1, len(distances.flatten()))
  ])

  return recommended_books

In [9]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [('Blue Diary', np.float32(0.71828747)), ('The Lovely Bones: A Novel', np.float32(0.7234864)), ('I Know This Much Is True', np.float32(0.7677075)), ('The Surgeon', np.float32(0.7699411)), ('The Weight of Water', np.float32(0.77085835))]]
You haven't passed yet. Keep trying!
